In [2]:
from dbfread import DBF
import pandas as pd
from simpledbf import Dbf5
from datetime import datetime
import numpy as np

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [17]:
def remover(df, colunas):
    return df.drop(columns=colunas, errors='ignore')

In [18]:
def binario(colunaA):
    empty = []
    colunaA = colunaA.to_list()  # Converte a coluna para uma lista, se necessário
    for valor in colunaA:
        if valor in [1, 2]:  # Verifica se o valor é 1 ou 2
            empty.append(1)
        else:
            empty.append(0)
    return empty

In [19]:
def extrair(colunaA):
    empty = []
    
    for stra in colunaA:
        if pd.isna(stra):  # Check for NaN values
            empty.append(np.nan)  # Use np.nan for missing values
            continue
        try:
            # Split the string and get the desired part
            stra_value = stra.split(" ")[1]  # Adjust index as needed
            empty.append(stra_value)  # Append the actual value
        except IndexError:
            # Handle case where split doesn't yield enough parts
            empty.append(np.nan)  # Or append None, depending on your needs
        except Exception as e:
            # Catch other potential errors
            print(f"Error processing entry '{stra}': {e}")
            empty.append(np.nan)

    return empty

In [20]:
def diferencaA(colunaA, colunaB):
    empty = []
    colunaA = colunaA.to_list()
    colunaB = colunaB.to_list()
    for i in range(len(colunaA)):
        date_format = "%Y-%m-%d"
        coluna_a = datetime.strptime(colunaA[i], date_format)
        coluna_b = datetime.strptime(colunaB[i], date_format)
        dif = (coluna_b - coluna_a).days
        if dif <= 30 :
            empty.append(0)
        elif dif <= 60 :
            empty.append(1)
        else:
            empty.append(2)
    return empty

In [21]:
def diferencaB(colunaA, colunaB):
    # Garantir que ambas as colunas sejam listas
    colunaA = colunaA.to_list()
    colunaB = colunaB.to_list()
    
    empty = []
    date_format = "%Y-%m-%d"  # Definindo o formato da data uma vez
    
    for date_a, date_b in zip(colunaA, colunaB):
        if pd.isna(date_a) or pd.isna(date_b):
            empty.append('Não Tratou')
            continue

        coluna_a = datetime.strptime(date_a, date_format)
        coluna_b = datetime.strptime(date_b, date_format)
        dif = (coluna_b - coluna_a).days

        if dif <= 30:
            empty.append(0)
        elif dif <= 60:
            empty.append(1)
        elif dif <= 90:
            empty.append(2)
        else:
            empty.append('Não Tratou')

    return empty

In [7]:
dbfile = Dbf5('pacigeral_jun24.dbf', codec='latin-1')
dbfile.to_csv("paciente_geral_v2.csv")
pd.set_option('display.max_columns', None)


In [8]:
df = pd.read_csv('paciente_geral_v2.csv', encoding='latin-1', sep=',')

C:\Users\silva\AppData\Local\Temp\ipykernel_12184\4039390663.py:1: DtypeWarning: Columns (37,38,39,40,77,78,79,82,93,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('paciente_geral_v2.csv', encoding='latin-1', sep=',')


In [9]:
df_ajustado = df[df['TOPOGRUP'] == 'C34']
df_ajustado.shape[0]


56800

In [10]:
df_ajustado = df_ajustado[df_ajustado['UFRESID'] == 'SP']
df_ajustado.shape[0]

52205

In [11]:
df_ajustado = df_ajustado[df_ajustado['BASEDIAG'] == 3] 
df_ajustado.shape[0]

50583

In [12]:
df_ajustado = df_ajustado[(df_ajustado['ECGRUP'] != 0) & (df_ajustado['ECGRUP'] != 'X') & (df_ajustado['ECGRUP'] != 'Y')]
df_ajustado.shape[0]

45452

In [13]:
df_ajustado = df_ajustado[(df_ajustado['HORMONIO'] != 1) & (df_ajustado['TMO'] != 1)]
df_ajustado.shape[0]

45241

In [14]:
df_ajustado = df_ajustado[df_ajustado['ANODIAG'] <= 2019]
df_ajustado.shape[0]

38771

In [15]:
df_ajustado = df_ajustado[df_ajustado['IDADE'] >= 20]
df_ajustado.shape[0]


38759

In [22]:
df_ajustado['CONSDIAG'] = diferencaA(df_ajustado['DTCONSULT'], df_ajustado['DTDIAG'])
df_ajustado.shape[0]

38759

In [23]:
df_ajustado['DIAGTRAT'] = diferencaB(df_ajustado['DTDIAG'], df_ajustado['DTTRAT'])
df_ajustado.shape[0]

38759

In [24]:
df_ajustado['TRATCONS'] = diferencaB(df_ajustado['DTCONSULT'], df_ajustado['DTTRAT'])
df_ajustado.shape[0]

38759

In [25]:
df_ajustado['DRS'] = extrair(df_ajustado['DRS'])
df_ajustado.shape[0]

38759

In [26]:
df_ajustado['DRSINST'] = extrair(df_ajustado['DRSINST'])
df_ajustado.shape[0]

38759

In [27]:
df_ajustado['OBITO'] = binario(df_ajustado['ULTINFO'])
df_ajustado.shape[0]

38759

In [28]:
df_ajustado = remover(df_ajustado,["UFNASC", "UFRESID", "CIDADE", "DTCONSULT", "CLINICA", "DTDIAG",
"BASEDIAG", "TOPOGRUP", "DESCTOPO", "DESCMORFO", "T", "N",
"M", "PT", "PN", "PM", "S", "G", "LOCALTNM", "IDMITOTIC",
"PSA", "GLEASON", "OUTRACLA", "META01", "META02", "META03",
"META04", "DTTRAT", "NAOTRAT", "TRATAMENTO", "TRATHOSP",
"TRATFANTES", "TRATFAPOS", "HORMONIO", "TMO", "NENHUMANT",
"CIRURANT", "RADIOANT", "QUIMIOANT", "HORMOANT", "TMOANT",
"IMUNOANT", "OUTROANT", "HORMOAPOS", "TMOAPOS", "DTULTINFO",
"CICI", "CICIGRUP", "CICISUBGRU", "FAIXAETAR", "LATERALI",
"INSTORIG", "RRAS", "ERRO", "DTRECIDIVA", "RECNENHUM",
"RECLOCAL", "RECREGIO", "RECDIST", "REC01", "REC02",
"REC03", "REC04", "CID", "DSCCIDO", "HABILIT", "HABIT11",
"HABILIT1", "CIDADEH", "PERDASEG"])
df_ajustado.shape[0]

38759

In [32]:
df_ajustado.head(100)

,INSTITU,DSCINST,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,MORFO,CIDO,DESCIDO,EC,ECGRUP,NENHUM,CIRURGIA,RADIO,QUIMIO,IMUNO,OUTROS,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,DRS,IBGEATEN,DRSINST,RRASINST,CIDADEINST,HABILIT2,OBITO
169,20230,SANTA CASA DE ARARAS,9,73,1,3503307,9,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IB,I,0,1,0,1,0,0,1,0,0,0,0,0,3,0,0,0,2002,10,3503307,10,RRAS 14,ARARAS,1,0
171,20621,SANTA CASA DE ARARAQUARA,9,67,2,3529302,9,1,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IIIB,III,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,2004,03,3503208,03,RRAS 18,ARARAQUARA,1,1
172,18058,HOSP ANCHIETA FUNDACAO ABC,2,72,2,3548708,2,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,1,0,0,0,1,0,0,0,0,0,3,0,1,2,2016,01,3548708,01,RRAS 01,SAO BERNARDO DO CAMPO,1,0
173,19976,HOSP MAT CELSO PIERRO PUCC,9,65,2,3501608,9,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IB,I,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,2010,07,3509502,07,RRAS 15,CAMPINAS,1,1
174,18058,HOSP ANCHIETA FUNDACAO ABC,3,63,1,3548708,2,2,C340,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,0,1,0,1,1,0,0,0,0,0,3,0,0,2,2015,01,3548708,01,RRAS 01,SAO BERNARDO DO CAMPO,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,4,SOBECCAN,5,59,2,3543402,9,2,C349,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,0,1,0,0,1,0,0,0,0,0,3,0,0,0,2003,13,3543402,13,RRAS 13,RIBEIRAO PRETO,5,0
336,612374,ICESP,3,64,2,3534401,9,1,C349,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IA,I,0,1,0,0,0,0,1,0,0,0,0,0,4,0,Não Tratou,Não Tratou,2009,01,3550308,01,RRAS 06,SAO PAULO,2,0
337,20281,SANTA CASA DE LIMEIRA,3,67,1,3526902,9,2,C349,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,0,1,0,0,1,0,0,0,0,0,3,0,0,1,2012,10,3526902,10,RRAS 14,LIMEIRA,1,0
338,17174,HOSP MARIO COVAS,9,62,1,3548708,2,1,C349,82513,82513.0,ADENOCARCINOMA ALVEOLAR,IV,IV,0,0,0,0,0,1,1,0,0,0,0,0,3,0,1,1,2007,01,3547809,01,RRAS 01,SANTO ANDRE,1,0


In [31]:
df_ajustado.isnull().sum()

INSTITU       0
DSCINST       0
ESCOLARI      0
IDADE         0
SEXO          0
IBGE          0
CATEATEND     0
DIAGPREV      0
TOPO          0
MORFO         0
CIDO          0
DESCIDO       0
EC            0
ECGRUP        0
NENHUM        0
CIRURGIA      0
RADIO         0
QUIMIO        0
IMUNO         0
OUTROS        0
NENHUMAPOS    0
CIRURAPOS     0
RADIOAPOS     0
QUIMIOAPOS    0
IMUNOAPOS     0
OUTROAPOS     0
ULTINFO       0
CONSDIAG      0
TRATCONS      0
DIAGTRAT      0
ANODIAG       0
DRS           0
IBGEATEN      0
DRSINST       0
RRASINST      0
CIDADEINST    0
HABILIT2      0
OBITO         0
dtype: int64